In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#Read in the csv file and turn the ContractDate column to datetime object
commish = pd.read_csv('Bags Medical Devices Company- Commision Detail.csv', 
                      parse_dates=['ContractDate'])

In [ ]:
commish.describe()

In [ ]:
commish.head()

In [ ]:
commish.tail()

In [ ]:
commish.columns

In [20]:
commish.dtypes

ContractId                      int64
ContractYear                    int64
ContractDate           datetime64[ns]
PrimarySalesRep                object
SecondarySalesRep              object
ContractAmount                  int64
PrimaryCommission             float64
SecondaryCommission           float64
TotalCommission               float64
dtype: object

In [21]:
def tweak_commish(df):
    return(df
          .loc[:, ['ContractId', 'ContractYear', 'ContractDate', 
                   'PrimarySalesRep', 'SecondarySalesRep', 'ContractAmount']]
          )

def commission(df):
    #Function to compute commissions and Total Commission
    def primary(df_):
        #Function to compute primary commissions for SR
        return(df_
              .assign(PrimaryCommission = lambda x: x['ContractAmount'] * 0.1)
              )
    
    def secondary(df_):
        #Function to compute secondary commissions for SR
        if ['SecondarySalesRep' == 'No secondary sales rep']:
            df_.assign(SecondaryCommission = 0)
        else:
            df_.assign(SecondaryCommission = lambda x: x['ContractAmount'] * 0.05)
    
    def total(df_):
        #Function to create Total commission for contract
        return(df_
              .assign(TotalCommission = PrimaryCommission + SecondaryCommission)
              )
    
    return(df
          .assign(PrimaryCommission = primary,
                  SecondaryCommission = secondary,
                 TotalCommission = total))
   

In [34]:
#ChatGPT Code Fix
def commission(df):
    # Function to compute commissions and Total Commission
    def primary(df_):
        # Function to compute primary commissions for SR
        return df_.assign(PrimaryCommission=lambda x: x['ContractAmount'] * 0.1)

    def secondary(df_):
        # Function to compute secondary commissions for SR
        df_['SecondaryCommission'] = 0
        df_.loc[df_['SecondarySalesRep'] != 'No secondary sales rep', 'SecondaryCommission'] = df_['ContractAmount'] * 0.05
        return df_

    def total(df_):
        # Function to create Total commission for contract
        return df_.assign(TotalCommission=lambda x: x['PrimaryCommission'] + x['SecondaryCommission'])

    return (
        df.pipe(primary)
          .pipe(secondary)
          .pipe(total)
    )

In [23]:
commish = tweak_commish(commish)

commish

,ContractId,ContractYear,ContractDate,PrimarySalesRep,SecondarySalesRep,ContractAmount
0,1,2008,2008-01-04,Inez Sanchez,Fred Jones,37419
1,2,2008,2008-01-07,Brady Marshall,No secondary sales rep,80152
2,3,2008,2008-01-11,Brady Marshall,Laura Barber,58643
3,4,2008,2008-01-12,Mitchel Bogart,No secondary sales rep,22380
4,5,2008,2008-01-12,Fred Jones,Kyle Freeman,50651
...,...,...,...,...,...,...
564,565,2011,2011-12-18,Laura Barber,No secondary sales rep,22003
565,566,2011,2011-12-19,Mitchel Bogart,Amanda Smith,33771
566,567,2011,2011-12-24,Kyle Freeman,Sandy McKnight,16803
567,568,2011,2011-12-26,Mitchel Bogart,Amanda Smith,18603


In [35]:
commish = commission(commish)

commish

,ContractId,ContractYear,ContractDate,PrimarySalesRep,SecondarySalesRep,ContractAmount,PrimaryCommission,SecondaryCommission,TotalCommission
0,1,2008,2008-01-04,Inez Sanchez,Fred Jones,37419,3741.9,1870.95,5612.85
1,2,2008,2008-01-07,Brady Marshall,No secondary sales rep,80152,8015.2,0.00,8015.20
2,3,2008,2008-01-11,Brady Marshall,Laura Barber,58643,5864.3,2932.15,8796.45
3,4,2008,2008-01-12,Mitchel Bogart,No secondary sales rep,22380,2238.0,0.00,2238.00
4,5,2008,2008-01-12,Fred Jones,Kyle Freeman,50651,5065.1,2532.55,7597.65
...,...,...,...,...,...,...,...,...,...
564,565,2011,2011-12-18,Laura Barber,No secondary sales rep,22003,2200.3,0.00,2200.30
565,566,2011,2011-12-19,Mitchel Bogart,Amanda Smith,33771,3377.1,1688.55,5065.65
566,567,2011,2011-12-24,Kyle Freeman,Sandy McKnight,16803,1680.3,840.15,2520.45
567,568,2011,2011-12-26,Mitchel Bogart,Amanda Smith,18603,1860.3,930.15,2790.45


In [42]:
commish['PrimarySalesRep'].groupby(commish['ContractDate'].dt.year).value_counts()

ContractDate  PrimarySalesRep 
2008          Mitchel Bogart      17
              Fred Jones          16
              Willie Johnson      16
              Amanda Smith        14
              Inez Sanchez        13
              Brady Marshall      12
              Kelly Thomas        11
              Laura Barber         9
              Jim Jackson          8
              Ryan Leaf            8
              Sandy McKnight       7
              Mark Esposito        6
              Kyle Freeman         4
              Victoria Beckham     3
              Jason Willis         2
2009          Amanda Smith        15
              Mitchel Bogart      15
              Laura Barber        14
              Willie Johnson      13
              Fred Jones          12
              Jim Jackson         12
              Kelly Thomas         9
              Victoria Beckham     9
              Kyle Freeman         8
              Inez Sanchez         7
              Brady Marshall       6
       

In [44]:
pd.set_option('display.max_rows', None)

commish['SecondarySalesRep'].groupby(commish['ContractDate'].dt.year).value_counts()

ContractDate  SecondarySalesRep     
2008          No secondary sales rep    77
              Kyle Freeman              11
              Jason Willis               7
              Mark Esposito              7
              Sandy McKnight             7
              Kelly Thomas               5
              Laura Barber               5
              Fred Jones                 4
              Inez Sanchez               4
              Jim Jackson                4
              Amanda Smith               3
              Victoria Beckham           3
              Willie Johnson             3
              Brady Marshall             2
              Mitchel Bogart             2
              Ryan Leaf                  2
2009          No secondary sales rep    66
              Brady Marshall            13
              Jason Willis               8
              Victoria Beckham           8
              Willie Johnson             8
              Ryan Leaf                  7
              Lau

In [41]:
commish['TotalCommission'].groupby(commish['ContractDate'].dt.year).sum()

ContractDate
2008    847539.60
2009    686969.35
2010    553952.65
2011    657750.40
Name: TotalCommission, dtype: float64

In [51]:
commish.groupby(commish['ContractDate'].dt.year).value_counts(['PrimarySalesRep', 'SecondarySalesRep'])

ContractDate  PrimarySalesRep   SecondarySalesRep     
2008          Mitchel Bogart    No secondary sales rep    16
              Inez Sanchez      No secondary sales rep     9
              Amanda Smith      No secondary sales rep     9
              Fred Jones        No secondary sales rep     7
              Willie Johnson    No secondary sales rep     7
              Brady Marshall    No secondary sales rep     7
              Kelly Thomas      Kyle Freeman               5
                                No secondary sales rep     5
              Willie Johnson    Jim Jackson                4
              Fred Jones        Inez Sanchez               4
              Sandy McKnight    No secondary sales rep     4
              Ryan Leaf         No secondary sales rep     3
              Fred Jones        Victoria Beckham           3
              Laura Barber      No secondary sales rep     3
              Mark Esposito     No secondary sales rep     3
              Fred Jones      